In [1]:
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 

In [14]:
def create_series(control_points, y_range=(0, 1), date_range=('2022-01-01', '2022-12-31'), noise=0.05 ):
    # Convert date range to pandas datetime for calculations
    start_date, end_date = pd.to_datetime(date_range[0]), pd.to_datetime(date_range[1])

    # Calculate total days for normalization
    total_days = (end_date - start_date).days

    # Sort control points by x value
    control_points.sort(key=lambda point: point[0])
    xs, ys = zip(*control_points)

    # Create spline
    spline = interp1d(xs, ys, kind='quadratic')

    # Generate x values for each day in the date range
    x = np.linspace(0, 1, num=total_days)

    # Generate y values (spline)
    y = spline(x)

    y += np.random.normal(0, noise, size=y.shape)

    # Apply Min-Max scaling to y values
    y_min, y_max = np.min(y), np.max(y)
    y = (y - y_min) / (y_max - y_min)  # Min-Max scaling

    # Scale y values to provided y range
    y = y * (y_range[1] - y_range[0]) + y_range[0]

    # Convert normalized x values back to dates
    x_dates = [start_date + pd.Timedelta(days=val*total_days) for val in x]

    x_dates = [date.date() for date in x_dates]

    # Create a pandas Series with dates as the index
    spline_series = pd.Series(y, index=x_dates)

    return spline_series

In [26]:
def combine_series(series_list, series_names):
    df = pd.DataFrame()

    for i in range(len(series_list)):
        df[series_names[i]] = series_list[i]
        
    return df

In [27]:
date_range = ('2022-01-01', '2022-12-31')

y_range_1 = (0, 5)

control_points_1 = [(0, 0), (0.5, 0.9), (1, 1)]

series_1 = create_series(control_points_1, y_range_1)

In [28]:
fig = px.line(series_1)
fig.show()

In [29]:
y_range_2 = (0, 10)

control_points_2 = [(0, 0), (0.5, 0.1), (1, 1)]

series_2 = create_series(control_points_2, y_range_2)

fig = px.line(series_2)
fig.show()

In [30]:
df = combine_series([series_1, series_2], ["series_1", "series_2"])

df

,series_1,series_2
2022-01-01,0.197459,1.345415
2022-01-02,0.000000,1.396628
2022-01-03,0.225723,1.574375
2022-01-04,0.110379,1.307817
2022-01-05,0.071617,0.974690
...,...,...
2022-12-26,4.383218,9.465037
2022-12-27,4.337276,9.547108
2022-12-28,3.918849,8.813449
2022-12-29,4.299761,9.191212


In [32]:
fig = px.line(df)

fig.show()

In [33]:
def update_with_effect(primary_series, secondary_series, correlation):
    return primary_series + secondary_series * correlation

In [36]:
series_3 = update_with_effect(series_1, series_2, 0.2)

df = combine_series([series_1, series_2, series_3], ["series_1", "series_2", "series_3"])
fig = px.line(df)

fig.show()

In [40]:
y_range_3 = (0, 8)

control_points_3 = [(0, 0), (0.5, 0.8), (1, 0.1)]

series_3 = create_series(control_points_2, y_range_2)

df["series_1"] = update_with_effect(df["series_1"], series_3, -1)

fig = px.line(df)
fig.show()